In [1]:
import properties_searcher
import pandas as pd
import torch
import chemistry_vae_selfies
import importlib
import math
import torch.nn as nn


In [2]:
list_of_properties = []

input_dim = 25 ### Size of the latent space
num_of_cycles = 1000### The number of hyperparameter searches you want to do, change accordingly
counter = 0

In [3]:
my_file = pd.read_csv('./datasets/PropsQM9/listprops.csv', index_col=None)##The file you want to train on, should contain SMILES reps, latent space reps and properties
my_file.dropna()


for i in range(len(my_file)):

    DipoleIn = my_file['dipole_moment'][i]
    GapIn = my_file['energy_gap'][i]
    SMILESCodeIn = my_file['smiles'][i]
    list_of_properties.append([SMILESCodeIn, GapIn, DipoleIn])

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

folder_path = "./datasets/"
file_name = "SelectedSMILES_QM9.txt"

full_path = folder_path + file_name

selfies_list, selfies_alphabet, largest_selfies_len, smiles_list, smiles_alphabet, largest_smiles_len = chemistry_vae_selfies.get_selfie_and_smiles_encodings_for_dataset(full_path)

selfies_alphabet = ['[#Branch2]', '[Ring2]', '[Branch2]', '[=Branch2]', '[O]', '[=O]', '[=C]', '[=N]', '[#Branch1]', '[=Branch1]', '[nop]', '[N]', '[Branch1]', '[F]', '[#C]', '[#N]', '[Ring1]', '[C]']

--> Translating SMILES to SELFIES...


Finished translating SMILES to SELFIES.
selfies aplhabet: ['[#Branch2]', '[=C]', '[#C]', '[O]', '[Branch2]', '[#N]', '[#Branch1]', '[nop]', '[N]', '[=Branch1]', '[Ring1]', '[Ring2]', '[=O]', '[F]', '[C]', '[=N]', '[=Branch2]', '[Branch1]']
smiles aplhabet: ['4', 'C', '3', 'N', '#', '2', 'F', ')', '1', '(', '5', '=', 'O', ' ']


In [5]:
# define source file location
file_to_load =  "./saved_models_RNN/"
# training file name encoder
training_file_nameE = "300/E"
# training file name decoder
training_file_nameD = "300/D"
# load data
#load_data_trained = file_to_load + training_file_nameE
# Alphabet has 18 letters, largest molecule is 21 letters. (build this as an output function later ... )
largest_selfies_len_dataset = largest_selfies_len
largest_smiles_len_dataset = largest_smiles_len

#in_dimension = len(selfies_alphabet)*largest_selfies_len
in_dimension = len(smiles_alphabet)*largest_smiles_len

# load the trained encoder
vae_encoder = torch.load(file_to_load + training_file_nameE) #, map_location=torch.device(device="cpu"))
#print(vae_encoder)

# load the trained decoder
vae_decoder = torch.load(file_to_load + training_file_nameD) #, map_location=torch.device(device="cpu"))
#print(vae_decoder)

In [6]:
selfies_rep, latent_rep, props_used = properties_searcher.filter(list_of_properties, largest_selfies_len_dataset, selfies_alphabet, vae_encoder, vae_decoder)

selfies_list, selfies_alphabet, largest_selfies_len, smiles_list, smiles_alphabet, largest_smiles_len = chemistry_vae_selfies.get_selfie_and_smiles_encodings_for_dataset(full_path)


selfies_alphabet = ['[#Branch2]', '[Ring2]', '[Branch2]', '[=Branch2]', '[O]', '[=O]', '[=C]', '[=N]', '[#Branch1]', '[=Branch1]', '[nop]', '[N]', '[Branch1]', '[F]', '[#C]', '[#N]', '[Ring1]', '[C]']
train_size = round(len(latent_rep)*0.8)

one_hots =  properties_searcher.create_onehot_instance_many(selfies_rep, largest_selfies_len, selfies_alphabet)

energy_gaps = []
#start_time = time.time()                                                                                                                                                                               


latent_rep, log_var_rep = properties_searcher.gen_new_latent_space(one_hots, largest_selfies_len, selfies_alphabet, vae_encoder, vae_decoder)



latent_space_vectors_valid = [item[0].detach().squeeze(0) for item in latent_rep]
log_vars_valid = [item[0].detach().squeeze(0) for item in log_var_rep]
properties_training = [torch.tensor(property_vector) for property_vector in props_used]  # this will convert lists to tensors  

--> Translating SMILES to SELFIES...
Finished translating SMILES to SELFIES.
selfies aplhabet: ['[#Branch2]', '[=C]', '[#C]', '[O]', '[Branch2]', '[#N]', '[#Branch1]', '[nop]', '[N]', '[=Branch1]', '[Ring1]', '[Ring2]', '[=O]', '[F]', '[C]', '[=N]', '[=Branch2]', '[Branch1]']
smiles aplhabet: ['4', 'C', '3', 'N', '#', '2', 'F', ')', '1', '(', '5', '=', 'O', ' ']


In [7]:
print(len(latent_space_vectors_valid))
print(len(log_vars_valid))

130251
130251


In [8]:
def min_max_normalize(tensor):
    min_val = torch.min(tensor)
    max_val = torch.max(tensor)
    normalized_tensor = (tensor - min_val) / (max_val - min_val)
    return normalized_tensor

In [9]:
for i in range(len(properties_training)):
    energy_gaps.append(props_used[i][0])



log_var2 = [math.e ** (-exponent) for exponent in log_vars_valid]


#log_vars_valid_normalise = (log_var2 - log_var2[(log_var2).argmin()]) / (log_var2[(log_var2).argmax()] - log_var2[(log_var2).argmin()])

In [10]:
normalized_tensor_list = [min_max_normalize(tensor) for tensor in log_var2]


In [11]:
log_var2[1]

tensor([  870.1587,   541.6920,  1232.6119,  1406.4443,  3066.4688,  3538.2463,
          480.1863,  1702.2864,  3667.7627,   695.9597,  2861.7271,  1762.9312,
         4839.9648,  2551.9009,  3155.3718,  2501.1599,  1584.5022,  4314.9805,
        11398.8779,  1621.9802,  1687.1516,   638.3410,  1715.5951,  1948.5055,
         2966.8137])

In [12]:
len(normalized_tensor_list)

130251

In [13]:
log_var_rep_train = normalized_tensor_list[:train_size]
log_var_rep_test = normalized_tensor_list[train_size:]



latents_train = latent_space_vectors_valid[:train_size]
latents_test = latent_space_vectors_valid[train_size:]

props_train = energy_gaps[:train_size]
props_test = energy_gaps[train_size:]

In [14]:
train_latent_space_tensor, test_latent_space_tensor, train_properties_tensor, test_properties_tensor = properties_searcher.make_tensors(latents_train, latents_test, props_train, props_test)

log_var_rep_train = torch.stack(log_var_rep_train)
log_var_rep_test = torch.stack(log_var_rep_test)

train_latent_space_tensor = train_latent_space_tensor.view(-1, input_dim)
test_latent_space_tensor = test_latent_space_tensor.view(-1, input_dim)
log_var_rep_train_tensor = log_var_rep_train.view(-1, input_dim)
log_var_rep_test_tensor = log_var_rep_test.view(-1, input_dim)


print(log_var_rep_train_tensor.size())
print(train_latent_space_tensor.size())

train_data_with_variances = torch.cat((train_latent_space_tensor, log_var_rep_train_tensor), dim=1)

torch.Size([104201, 25])
torch.Size([104201, 25])


In [38]:
print(log_var_rep_test_tensor[1])

tensor([0.0255, 0.0000, 0.6345, 0.1347, 0.1470, 0.2773, 0.0462, 0.2967, 0.3003,
        0.1171, 1.0000, 0.5250, 0.5243, 0.2843, 0.6753, 0.4109, 0.1829, 0.5580,
        0.0815, 0.0795, 0.2203, 0.0371, 0.1314, 0.2583, 0.6376])


In [15]:
print(len(train_data_with_variances))

104201


In [16]:
print(len(log_var_rep_train_tensor))
print(len(train_latent_space_tensor))

104201
104201


In [17]:
print(len(log_var_rep_train_tensor))

104201


In [40]:
importlib.reload(properties_searcher)


<module 'properties_searcher' from '/users/sgccook3/DM_chems/VAE_sandbox/updated_juri/properties_searcher.py'>

In [41]:
for i in range(num_of_cycles):
            # Sample input dimensions for 100,000 compounds represented by 25 numbers
        hidden_dim = 1024 #random.choice([512, 1024, 2048])  # You can increase the hidden dimensions for more complexity
        prop_hidden_dim = int(hidden_dim/2)  # You can adjust this based on the complexity of the task
        prop_pred_activation = 'leaky_relu' #random.choice(['relu', 'leaky_relu'])
        prop_pred_dropout = 0#random.choice([0, 0.1, 0.2])
        prop_pred_depth = 6#random.choice([4,5,6])
        prop_growth_factor = 0.4#random.choice([0.4, 0.5, 0.6])
        lr_random = 0.0005#random.choice([0.001, 0.0005, 0.0001])
        epochs = 2





        # Instantiate the model with hyperparameters
        model = properties_searcher.PropertyRegressionModel(input_dim, hidden_dim, prop_hidden_dim, prop_pred_activation, prop_pred_dropout, prop_pred_depth, prop_growth_factor)

        # Define your loss function (e.g., Mean Squared Error)
        loss_function = nn.MSELoss()

        # Define your optimizer (e.g., Adam)
        optimizer = torch.optim.Adam(model.parameters(), lr=lr_random)   
        decay_step_size = 100  # Adjust this as per your preference
        decay_gamma = 0.9     # Adjust this as per your preference
        scheduler = properties_searcher.StepLR(optimizer, step_size=decay_step_size, gamma=decay_gamma)


        batch_size = 15000  # You can adjust this based on your hardware and dataset size

        # Split the training data and properties into batches
        train_data_batches = train_latent_space_tensor.split(batch_size)
        train_props_batches = train_properties_tensor.split(batch_size)
        latent_variances_batch = log_var_rep_train_tensor.split(batch_size)
        print(len(latent_variances_batch[2][1]))


        


###

        for epoch in range(epochs):

            optimizer.zero_grad()
            predictions = model(train_latent_space_tensor, log_var_rep_train_tensor)
            loss = loss_function(predictions.squeeze(), train_properties_tensor)
            loss.backward()
            optimizer.step()
            scheduler.step()

            #end_time = time.time()
            #epoch_duration = end_time - start_time

            #print(f"Epoch {epoch}/{epochs} - Duration: {epoch_duration:.2f} seconds")

            total_loss = 0.0

            # Iterate through batches
            for data_batch, props_batch, var_batch in zip(train_data_batches, train_props_batches, latent_variances_batch):

                # Concatenate the latent space tensor and variances tensor for this batch
                data_with_variances_batch = torch.cat((data_batch, var_batch), dim=1)

                optimizer.zero_grad()
                predictions = model(data_batch,var_batch)  # Pass the combined tensor as input
                loss = loss_function(predictions.squeeze(), props_batch)
                loss.backward()
                optimizer.step()

                total_loss += loss.item()

                

                # Update the learning rate
            scheduler.step()

                # Calculate average loss for the epoch
            avg_loss = total_loss / len(train_data_batches)



###

        with torch.no_grad():
            test_predictions = model(test_latent_space_tensor, log_var_rep_test_tensor)
            #print('test_predictions length:',len(test_predictions))
            #print('actual predictions length:',len(props_test))
            #test_loss = loss_function(test_predictions.squeeze(), test_properties_tensor)
            #print(f"Test Loss: {test_loss.item():.4f}")

###


        # Generate predictions for the test set
        y_pred = test_predictions
        y_test = props_test

        mse, mae, r2 = properties_searcher.stats(y_pred, y_test)
        print(r2)

25


torch.Size([104201, 512])
torch.Size([15000, 512])
torch.Size([15000, 512])
torch.Size([15000, 512])
torch.Size([15000, 512])
torch.Size([15000, 512])


KeyboardInterrupt: 

In [37]:
print(len(test_latent_space_tensor))

26050


In [ ]:
log_var_rep_train_tensor.size()

NameError: name 'log_var_rep_train_tensor' is not defined